In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Criando dataset simulado
np.random.seed(42)
n = 1000

dados_qualidade = pd.DataFrame({
    "temperatura": np.random.normal(200, 15, n),
    "pressao": np.random.normal(5, 1.2, n),
    "tempo_operacao": np.random.normal(10, 2, n),
    "tipo_material": np.random.choice(["A", "B", "C"], n),
    "turno": np.random.choice(["Manhã", "Tarde", "Noite"], n),
})

# Regra para gerar rótulo de aprovação (1 = aprovado, 0 = reprovado)
# A aprovação é mais provável se a temperatura e pressão estiverem dentro de faixas ideais e não for material C.
dados_qualidade["aprovado"] = (
    (dados_qualidade["temperatura"] > 190) &
    (dados_qualidade["pressao"] > 4.5) &
    (dados_qualidade["tempo_operacao"] < 12) &
    (dados_qualidade["tipo_material"] != "C")
).astype(int)

# Pré-processar os dados
# Codificação One-Hot para variáveis categóricas
dados_qualidade = pd.get_dummies(dados_qualidade, columns=['tipo_material', 'turno'], drop_first=True)

# Variáveis Preditivas (X) e Variável Alvo (y)
X = dados_qualidade.drop("aprovado", axis=1)
y = dados_qualidade["aprovado"]

# 3. Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Lista de modelos a serem comparados
modelos = {
    "Regressão Logística": LogisticRegression(solver='liblinear', random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

resultados_qualidade = {}

# Treinar e avaliar os modelos
print("Análise de Classificação para Aprovação de Componentes\n")

for nome, modelo in modelos.items():
    # Treinamento do modelo
    modelo.fit(X_train, y_train)

    # Previsão
    y_pred = modelo.predict(X_test)

    # Cálculo das métricas
    acuracia = accuracy_score(y_test, y_pred)
    precisao = precision_score(y_test, y_pred, zero_division=0)
    revocacao = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    resultados_qualidade[nome] = {
        "Acurácia": acuracia,
        "Precisão": precisao,
        "Recall": revocacao,
        "F1-Score": f1
    }

    print(f"--- Modelo: {nome} ---")

    # Matriz de Confusão:
    # [[Verdadeiro Negativo, Falso Positivo]
    #  [Falso Negativo, Verdadeiro Positivo]]
    matriz = confusion_matrix(y_test, y_pred)
    print("Matriz de Confusão:")
    print(matriz)

    # Exibição das métricas
    print(f"\nAcurácia: {acuracia:.4f}")
    print(f"Precisão: {precisao:.4f}")
    print(f"Recall (Revocação): {revocacao:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("\n" + "="*40 + "\n")

# Comparação e Exibição Consolidada
df_resultados_qualidade = pd.DataFrame(resultados_qualidade).T

print("Comparação de Desempenho dos Modelos:")
print(df_resultados_qualidade.sort_values(by='Acurácia', ascending=False))